***Проанализировать активность пользователей платформы и рассчитать метрики:***  
- retention  
- rolling retention  
- lifetime  
- churn rate  
- mau  
- wau  
- dau  

### Подготовка данных

In [1]:
import pandas as pd

In [2]:
df_reg = pd.read_csv('registrations.csv', sep = ';', parse_dates = ['registration_date'])
df_ent = pd.read_csv('entrie.csv', sep = ';', parse_dates = ['entry_date'])

df = pd.merge(df_reg, df_ent, on='user_id', how='inner')
df['diff'] = df['entry_date'] - df['registration_date']

df.head()

,user_id,registration_date,entry_date,diff
0,1,2021-01-01,2021-01-01,0 days
1,1,2021-01-01,2021-01-03,2 days
2,1,2021-01-01,2021-01-04,3 days
3,1,2021-01-01,2021-01-04,3 days
4,1,2021-01-01,2021-01-05,4 days


### Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе.

Cохраните результат в переменную retention_15_day

Примечание: результат округлите до 5 знаков после запято

In [3]:
january_cohort = df_reg[df_reg['registration_date'] <= '2021-01-31']
return_15 = df[(df['diff'] == '15 days') & (df['registration_date'] <= '2021-01-31')]

In [4]:
retention_15_day = round(return_15['user_id'].nunique()/january_cohort['user_id'].nunique()*100, 5)
retention_15_day

54.65116

### Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты.

Сохраните результат в переменную rolling_retention

Примечание: результат округлите до 5 знаков после запятой

In [5]:
january_cohort = df_reg[df_reg['registration_date'] <= '2021-01-31']
return_30 = df[(df['diff'] >= '30 days') & (df['registration_date'] <= '2021-01-31')]

In [6]:
rolling_retention = round(return_30['user_id'].nunique()/january_cohort['user_id'].nunique()*100, 5)
rolling_retention

29.06977

### Посчитайте Lifetime по всем пользователям как интеграл от n-day retention.

Сохраните результат в переменную lifetime

Примечание: результат округлите до 5 знаков после запятой

In [7]:
n_day_retention = df.groupby(['diff'])['user_id'].agg('nunique').reset_index()
n_day_retention['div'] = n_day_retention['user_id'] / n_day_retention.iloc[0]['user_id']
n_day_retention

,diff,user_id,div
0,0 days,1000,1.000
1,1 days,466,0.466
2,2 days,471,0.471
3,3 days,472,0.472
4,4 days,477,0.477
5,5 days,466,0.466
6,6 days,473,0.473
7,7 days,482,0.482
8,8 days,494,0.494
9,9 days,483,0.483


In [8]:
lifetime = round(n_day_retention['div'].sum(), 5)
lifetime

14.804

### Посчитайте Churn rate 29 дня (в долях) по всем пользователям.

Сохраните результат в переменную churn_29.

In [9]:
return_29 = df[df['diff'] >= '29 days']
churn_29 = (len(df_reg) - return_29['user_id'].nunique()) / len(df_reg)
churn_29

0.509

### Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей.

Сохраните результат в переменные dec_mau, dec_wau, dec_dau соответственно

Примечание: последний месяц записей - декабрь. Поэтому mau рассчитываем для декабря (2021 года), для wau берем последнюю неделю - с 25 по 31 декабря, и для dau соответственно последний день - 31 декабря.

In [10]:
last_month = df_ent[(df_ent['entry_date'] >= '2021-12-01') & (df_ent['entry_date'] <= '2021-12-31')]
dec_mau = last_month['user_id'].nunique()
dec_mau

133

In [11]:
last_week = df_ent[(df_ent['entry_date'] >= '2021-12-25') & (df_ent['entry_date'] <= '2021-12-31')]
dec_wau = last_week['user_id'].nunique()
dec_wau

84

In [12]:
last_day = df_ent[df_ent['entry_date'] == '2021-12-31']
dec_dau = last_day['user_id'].nunique()
dec_dau

47

### Посчитайте Mau, Wau, Dau усредненные.

Сохраните результат в переменные avg_mau, avg_wau, avg_dau соответственно

Примечание: результаты округлите до 5 знаков после запятой

In [13]:
df_ent['month'] = df_ent['entry_date'].dt.month
df_ent['week'] = df_ent['entry_date'].dt.isocalendar().week
df_ent

,user_id,entry_date,month,week
0,1,2021-01-01,1,53
1,2,2021-01-01,1,53
2,3,2021-01-01,1,53
3,4,2021-01-01,1,53
4,5,2021-01-01,1,53
...,...,...,...,...
20729,990,2021-12-31,12,52
20730,990,2021-12-31,12,52
20731,979,2021-12-31,12,52
20732,979,2021-12-31,12,52


In [14]:
grouped_month = df_ent.groupby(['month'])['user_id'].agg('nunique').reset_index()
avg_mau = round(grouped_month['user_id'].sum()/len(grouped_month),5)
avg_mau

102.58333

In [15]:
grouped_week = df_ent.groupby(['week'])['user_id'].agg('nunique').reset_index()
avg_wau = round(grouped_week['user_id'].sum()/len(grouped_week), 5)
avg_wau

89.86792

In [16]:
grouped_day = df_ent.groupby(['entry_date'])['user_id'].agg('nunique').reset_index()
avg_dau = round(grouped_day['user_id'].sum()/len(grouped_day), 5)
avg_dau

40.5589